In [ ]:
import numpy as np
import math
from queue import PriorityQueue

In [ ]:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """

    def __init__ (self, state,hval,parent = None):
        
        """
        Initializes the PuzzleNode with the nodes' state, hval,and points towards its parent
        
        Parameters:
            A list of lists: List
            State: List
            hval: Integer
            parent: List
        ----------------
        Methods:
            move: List
                Provides a list of valid moves
            generate_child: List
                Using legal moves, produces children of the current node
            inv_count: int
                Counts the number of inversions for the current state
            is_solvable: boolean
                Tells if the puzzle is solvable or no
            goal_state: List
                Return a list which is the goal state of the puzzle
            is_solution: Boolean
                Checks if the goals state has been reached
            valid: Boolean
                Checks if the inputs are valid
        """
        
        self.orig_state = state
        #The puzzle, list of list, is converted into a flat list (a single list) to make element manipulation easier
        self.state = np.array([state]).flatten()
        self.parent = parent
        self.pruned = False
        self.pos_zero = None
        self.hval = hval

        
        #The gval is basically the depth level, so if the parent is none that means that we are at depth level 0 and if not
        #then we would just add 1 to it
        if self.parent is None:
            self.gval = 0
        else:
            self.gval = self.parent.gval+1
         
        #Calculates the fval by just adding the gval and hval
        self.fval = self.gval + self.hval
        
        #Calculates the position of 0 as it would allow us to calculate legal moves in the future
        if self.pos_zero ==None:
            self.pos_zero = list(np.where(self.state==0))[0]
        
    
    #this method is defined so that the priority queue is formed based on the evaluation function of the node
    
    def __lt__(self,other):
        
        return self.fval <= other.fval

    def __str__(self):
        return str(self.state)
    
    
    # The function first helps us locate the blank space
    # and based on that tells us the legal actions that can be takes. Returns a list of legal moves 

    def move(self):
        moves = ['L','R','U','D']
        
        x = self.pos_zero
        n = int(math.sqrt(len(self.state)))
        
        
        if x % n == 0:
            moves.remove('L')
        if x % n == n-1:
            moves.remove('R')
        if x - n < 0:
            moves.remove('U')
        if x + n > (n*n) - 1:
            moves.remove('D')

        return moves
    
    #function to make successor states. Would be based on the result of function moves. For generalization I used
    #dimensions as it would allow us to make legal children of any n*n puzzle.
    def generate_child(self):
        children=[]
        x = self.pos_zero
        potential_moves = self.move()
        dimension = int(math.sqrt(len(self.state)))
        
        for action in potential_moves:
            new_state = self.state.copy()
            if action == 'U':
                new_state[x], new_state[x-dimension] = new_state[x-dimension], new_state[x]
            elif action == 'D':
                new_state[x], new_state[x+dimension] = new_state[x+dimension], new_state[x]
            elif action == 'L':
                new_state[x], new_state[x-1] = new_state[x-1], new_state[x]
            elif action == 'R':
                new_state[x], new_state[x+1] = new_state[x+1], new_state[x]
            children.append(PuzzleNode(new_state,0,self))
        return children    


    
    #Counts the number of inversions for the initial puzzle
    def inv_count(self):
        counter = 0
        list_state = list(self.state)
        len_ = len(list_state)
        for i in range(len_):
            for j in range(i+1,len_):
                if (list_state[j] and list_state[i] and list_state[i]> list_state[j]):
                    counter+=1
        return counter
    
    
    #A generalizable function to check if the solution is possible or no
    #Inspiration taken from https://www.geeksforgeeks.org/check-instance-15-puzzle-solvable/
    def is_solvable(self):
        
        inv_count = self.inv_count()
        len_ = len(self.orig_state)
        pos_row = int(self.pos_zero)/len_

        if len_%2==1:
            
            return inv_count%2==0
        
        else:
            if pos_row%2==0 and inv_count%2==0:
                
                return True
            elif pos_row %2==1 and inv_count%2==1:
                return True
            else:
                return False

    #Checks what the goal state is for the puzzle
    def goal_state(self):
        states = list(np.array([self.state]).flatten())
        n = int(math.sqrt(len(states)))
        goal = [i for i in range(0,n*n)]
        return goal
    
    #Compares the goal state with the current state to see if we have reached a solution
    def is_solution(self):
        
        goal= self.goal_state()
        if list(self.state) == goal:
            return True
        return False
    
    #Checks if the initial puzzle state is valid or not. Checks if the values range from 0 to n and that the dimensison
    #are all consistent: are n * n.
    def valid(self):
        len_ = len(self.orig_state)
        if sorted(self.state) == [i for i in range(0,len_*len_)] and np.array(self.orig_state).shape == (len_,len_):
            return True
        return False


In [ ]:
def memoization(function):
    """
        A function that memoizases h values and returns the cached 
        result. Prevents recalculating redundant values
        Input:
            -function: a heuristic function h1, h2
        Output:
            -memo_dic[key_value]: h cost value
            -wrapper: a function
    """
    
    memo_dic = {}
    
    
    def wrapper(*args):
        #make a hashable string
        key_value = str(args[0])
        
        #add new key if not in dict already
        if key_value not in memo_dic:
            memo_dic[key_value] = function(args[0])
            #Printed the dictionary at each iteration to check if it works or no
            #print(memo_dic)
 
        return memo_dic[key_value]
    
    return wrapper


# Misplaced tiles heuristic
@memoization
def h1(state):

    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    
    states = list(np.array([state]).flatten())
    n = int(math.sqrt(len(states)))
    counter = 0;
    
    goal = [i for i in range(0,n*n)]
    
    
    for i,j in zip(states,goal):
        if i ==0:
            continue
        if i!= j:
            counter+=1
             
    return counter

# Manhattan distance heuristic
@memoization
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    heuristic=0
    states = list(np.array([state]).flatten())
    n = int(math.sqrt(len(states)))
    goal = [i for i in range(0,n*n)]
    
    cost =sum(abs(b%n - g%n) + abs(b//n - g//n)
            for b, g in ((states.index(i), goal.index(i)) for i in range(1, n*n)))
    heuristic+= cost 
    return heuristic
heuristics = [h1,h2]

In [ ]:
ef solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    dimension = len(state)
    
    exp = 0
    max_frontier = 0
    #The cost_database helps us keep track of the state and their corresponding children. The keys are the parents
    #and the values are the children of that state.
    cost_database = {}
    explored= []
    opt_path = None
    err = 0
    steps = 0
    
    
    initial_node = PuzzleNode(state,0)
    
    
    #Returns -1 if the state is inconsistent.
    if not initial_node.valid():
        err = -1
    #Returns -2 if the state is not_solvable. This was part of the extension question
    elif not initial_node.is_solvable():
        err = -2
    
    else:
        frontier = PriorityQueue()
        frontier.put(initial_node)


        while not frontier.empty():
            

            current_node = frontier.get()
            
            exp+=1
            
            
            if current_node.pruned:
                continue
            explored.append(list(current_node.state))

            if list(current_node.state) == current_node.goal_state():
                break

            children = current_node.generate_child()

            for child in children:
                
                #tentative cost of the node
                gval = child.gval 
                
                #Checks and prunes the nodes so that we dont have to visit them again and again.
                if tuple(child.state) in cost_database:
                    if cost_database[tuple(child.state)].gval>gval:
                        cost_database[tuple(child.state)].pruned = True
                    else:
                        continue
                hval = heuristic(tuple(child.state))

                next_node = PuzzleNode(child.state,hval,current_node)
                frontier.put(next_node)

                if frontier.qsize() > max_frontier:
                    max_frontier = frontier.qsize()
                cost_database[tuple(child.state)] = next_node


            
        
        #This block of code helps us reconstruct the path by looking at the parent of the last child. So it backtracks from the
        #lost child node to the top most node: the parent
        previous_parent = current_node.parent
        opt_path = []
        
        
        opt_path.append([list(i) for i in np.array_split(current_node.state,dimension)])
        while previous_parent:

            opt_path.append([list(i) for i in np.array_split(previous_parent.state,dimension)])
            previous_parent = previous_parent.parent


        steps = len(opt_path)-1
        opt_path = opt_path[::-1]
    return steps,exp,max_frontier,opt_path,err 

